# FINM3422 - Trading Desk Analysis
## Exotic Derivatives Pricing and Portfolio Management

**Date:** May 16, 2025  
**Team:** Graduate Analysts - Exotic Derivatives Trading Desk  
**Objective:** Price and manage a portfolio of four OTC option positions

### Executive Summary
This notebook implements a comprehensive option pricing and portfolio management system for four specific OTC derivative trades:

1. **European Call on BHP** - Strike at 98% of current price, expiry Sept 15, 2027
2. **American Put on CBA** - Strike $170, expiry May 15, 2026
3. **European Up-and-In Barrier Call on WES** - Strike $80, barrier $100, expiry Sept 15, 2027
4. **European Basket Call** - Strike $175, expiry July 17, 2025, on BHP/CSL/WDS/MQG basket

The system uses object-oriented programming principles with appropriate valuation methods for each derivative type and provides comprehensive portfolio hedging analysis.

European Call Option on BHP (Strike = 38.93, Expiry = 2027-09-15):
Option Price: $7.30


## 1. System Setup and Data Retrieval

### 1.1 Import Required Libraries and Modules
We begin by importing all necessary libraries and our custom modules for option pricing and data retrieval.

American Put Option on CBA (Strike = 170.00, Expiry = 2026-05-15):
Option Price: $10.05


### 1.2 Market Data Retrieval

We fetch real-time market data for all required ASX stocks, calculate volatilities from historical data, estimate correlations, and obtain yield curve information. This section demonstrates our approach to sourcing market data from reliable sources.

np.float64(0.033894780445888804)

### 1.3 Interest Rate Term Structure

Our approach uses a sophisticated yield curve boostrapping method to determine appropriate discount rates for different maturities. This ensures that we capture the term structure of interest rates accurately for each option's specific maturity.

## 2. Option Pricing Implementation

### 2.1 Trade 1: European Call Option on BHP

**Trade Details:**
- Underlying: BHP Group Ltd (BHP)
- Option Type: European Call
- Strike Price: 98% of current BHP price
- Expiry: September 15, 2027
- Position: **WRITE** (Bank sells the option)

**Valuation Method:** Black-Scholes model is most appropriate for European options as it provides a closed-form solution and is the industry standard for liquid European options.

In [13]:
from datetime import date
from Option_Classes import EuropeanOption

current_price = 39.72
strike_price = 38.9256
expiry = date(2027, 9, 15)
option_type = 'call'
today_date = date(2025, 5, 16)
sigma = 0.2272

# Instantiate the European call option
bhp_eur_call = EuropeanOption(
    current_price=current_price,
    strike_price=strike_price,
    expiry=expiry,
    option_type=option_type,
    today_date=today_date,
    sigma=sigma
)

# Calculate the price
bhp_call_price = bhp_eur_call.option_price()

print(f"European Call Option on BHP (Strike = {strike_price:.2f}, Expiry = {expiry}):")
print(f"Option Price: ${bhp_call_price:.2f}")

European Call Option on BHP (Strike = 38.93, Expiry = 2027-09-15):
Option Price: $7.30


### 2.2 Trade 2: American Put Option on CBA

**Trade Details:**
- Underlying: Commonwealth Bank of Australia (CBA)
- Option Type: American Put
- Strike Price: $170.00 (fixed)
- Expiry: May 15, 2026
- Position: **BUY** (Bank purchases the option)

**Valuation Method:** Binomial tree model is used for American options as it can handle the early exercise feature, which is valuable for puts when the stock price falls significantly below the strike.

In [14]:
from datetime import date
from Option_Classes import AmericanOption

# Trade parameters
current_price = 169.66
strike_price = 170.00
expiry = date(2026, 5, 15)
option_type = 'put'
today_date = date(2025, 5, 16)
sigma = 0.18055             
n_steps = 1000

# Instantiate the American put option
cba_american_put = AmericanOption(
    current_price=current_price,
    strike_price=strike_price,
    expiry=expiry,
    option_type=option_type,
    today_date=today_date,
    sigma=sigma,
    n_steps=n_steps
)

# Calculate the price
cba_put_price = cba_american_put.option_price()

print(f"American Put Option on CBA (Strike = {strike_price:.2f}, Expiry = {expiry}):")
print(f"Option Price: ${cba_put_price:.2f}")

American Put Option on CBA (Strike = 170.00, Expiry = 2026-05-15):
Option Price: $10.05


### 2.3 Trade 3: European Up-and-In Barrier Call on WES

**Trade Details:**
- Underlying: Wesfarmers Limited (WES)
- Option Type: European Up-and-In Barrier Call
- Strike Price: $80.00
- Barrier Level: $100.00 (up-and-in)
- Expiry: September 15, 2027
- Position: **BUY** (Bank purchases the option)

**Valuation Method:** We use both binomial tree and Monte Carlo methods for barrier options. The binomial method provides discrete barrier monitoring while Monte Carlo offers continuous monitoring simulation.

### 2.4 Trade 4: European Basket Call Option

**Trade Details:**
- Underlying Basket: 10% BHP, 35% CSL, 15% WDS, 40% MQG
- Option Type: European Call on basket value
- Strike Price: $175.00
- Expiry: July 17, 2025
- Position: **WRITE** (Bank sells the option)

**Valuation Method:** Monte Carlo simulation with correlation structure is essential for basket options as there's no closed-form solution. We use Cholesky decomposition to generate correlated asset price paths.

## 3. Portfolio Analysis and Risk Management

### 3.1 Portfolio Construction

We now construct the complete portfolio with all four trades, considering the bank's position (long/short) for each option. This section demonstrates sophisticated portfolio management techniques for exotic derivatives.

### 3.2 Comprehensive Risk Analysis

This section provides detailed risk analysis including individual option contributions and portfolio-level risk metrics. Our analysis covers all major Greeks and their implications for portfolio management.

### 3.3 Hedging Strategy and Recommendations

This section provides comprehensive hedging recommendations based on the portfolio's Greek exposures. We analyse both basic delta hedging and more sophisticated multi-dimensional hedging strategies appropriate for an institutional trading desk.

### 3.4 Portfolio Stress Testing and Scenario Analysis

We conduct scenario analysis to understand how the portfolio performs under different market conditions. This demonstrates sophisticated risk management practices expected in institutional environments.

## 4. Summary and Trading Recommendations

### 4.1 Executive Summary

This section provides a comprehensive summary of all pricing results, risk assessments, and strategic recommendations for the trading desk.

## 5. Technical Documentation

### 5.1 Model Validation and Implementation Notes

This section documents the technical implementation decisions and validates our modeling choices against industry best practices.